## Local Inference on GPU
Model page: https://huggingface.co/google/gemma-7b-it

⚠️ If the generated code snippets do not work, please open an issue on either the [model repo](https://huggingface.co/google/gemma-7b-it)
			and/or on [huggingface.js](https://github.com/huggingface/huggingface.js/blob/main/packages/tasks/src/model-libraries-snippets.ts) 🙏

In [ ]:
!pip install -U llama-cpp-python pandas openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.7/50.7 MB 53.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 133.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 5.0 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.3.16-cp312-cp312-linux_x86_64.whl size=4500226 sha256=50e7406bc26401f92e7956715fdc48ce45d46dbe510df1e1ff10f946fd7e990a
  Stored in directory: /root/.cache/pip/wheels/90/82/ab/8784ee3fb99ddb07fd36a679ddbe63122cc07718f6c1eb3be8
Successfully built llama-cpp-python
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency

\The model you are trying to use is gated. Please make sure you have access to it by visiting the model page.To run inference, either set HF_TOKEN in your environment variables/ Secrets or run the following cell to login. 🤗

In [ ]:
from huggingface_hub import login
login(new_session=False)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# !pip install llama-cpp-python
import pandas as pd

from llama_cpp import Llama





In [ ]:
from google.colab import files
uploaded = files.upload()




Saving PV_Signal_List_10_TEST_SCENARIOS.xlsx to PV_Signal_List_10_TEST_SCENARIOS.xlsx


In [ ]:
import pandas as pd

df = pd.read_excel("PV_Signal_List_10_TEST_SCENARIOS.xlsx")

print(df.head())

       Signal term Date detected                                     Status  \
0           Stroke        May-25                                    Ongoing   
1              SJS        Jun-25                                    Refuted   
2      Myocarditis        Apr-25   Closed as Potential Risk (Not Important)   
3  QT prolongation        Sep-24       Closed as Potential Risk (Important)   
4     Pancreatitis        Mar-25  Closed as Identified Risk (Not Important)   

  Date closed     Source of signal  \
0         NaN        Meta-analysis   
1      25-Aug         Case reports   
2      10-Jul             Registry   
3      12-Oct       Clinical Study   
4      09-Sep  Spontaneous Reports   

                                          Summary              Action  
0                   Signal still under assessment             Pending  
1          Evaluation shows no causal association                 NaN  
2  Evidence suggests possible risk but low impact             Monitor  
3       

In [ ]:


llm = Llama.from_pretrained(
    repo_id="google/gemma-7b-it",
    filename="gemma-7b-it.gguf",
    n_ctx=4096,    # context length
    n_gpu_layers=-1,  # use GPU if available
)

./gemma-7b-it.gguf:   0%|          | 0.00/34.2G [00:00<?, ?B/s]

llama_model_loader: loaded meta data with 19 key-value pairs and 254 tensors from /root/.cache/huggingface/hub/models--google--gemma-7b-it/snapshots/9c5798d27f588501ce1e108079d2a19e4c3a2353/./gemma-7b-it.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = gemma
llama_model_loader: - kv   1:                               general.name str              = gemma-7b-it
llama_model_loader: - kv   2:                       gemma.context_length u32              = 8192
llama_model_loader: - kv   3:                          gemma.block_count u32              = 28
llama_model_loader: - kv   4:                     gemma.embedding_length u32              = 3072
llama_model_loader: - kv   5:                  gemma.feed_forward_length u32              = 24576
llama_model_loader: - kv   6:                 gemma.attention.head_count u32 

In [ ]:
# Define the system instruction (same as your LLaMA setup)
system_prompt = (
    "You are an expert in pharmacovigilance and ICH E2C(R2) guidelines. "
    "Your task is to read ONLY the input safety signal data exactly as provided "
    "and map each signal to the correct PBRER section(s) according to Appendix F. "
    "Do NOT infer medical judgment beyond what is explicitly stated.\n\n"

    "Use ONLY the following seven scenarios when categorizing signals:\n"
    "1. Ongoing Signal\n"
    "2. Closed + Refuted\n"
    "3. Closed → Potential Risk (NOT important)\n"
    "4. Closed → Potential Risk (IMPORTANT)\n"
    "5. Closed → Identified Risk (NOT important)\n"
    "6. Closed → Identified Risk (IMPORTANT)\n"
    "7. Closed → New Information on Known Risk\n\n"

    "Mapping logic:\n"
    "- Ongoing Signal → Section 15 only.\n"
    "- Closed + Refuted → Section 15 + Section 16.2.\n"
    "- Closed → Potential Risk (NOT important) → Section 15 + Section 16.2.\n"
    "- Closed → Potential Risk (IMPORTANT) → Section 15 + Section 16.2 + Section 16.4.\n"
    "- Closed → Identified Risk (NOT important) → Section 15 + Section 16.2.\n"
    "- Closed → Identified Risk (IMPORTANT) → Section 15 + Section 16.2 + Section 16.4.\n"
    "- Closed → New Information on Known Risk → Section 15 + Section 16.3 + Section 16.4.\n\n"

    "Very important rules:\n"
    "- If the dataset does NOT explicitly state identified/potential/important, do NOT infer it.\n"
    "- Do NOT perform causality assessment.\n"
    "- If status wording is unclear, categorize as Ongoing or Refuted only.\n"
    "- You MUST strictly restrict your reasoning to the information given in the dataset.\n"
    "- Do NOT hallucinate: do NOT invent risks, actions, importance level, severity, "
    "additional evidence, or clinical interpretations not present in the input.\n"
    "- Do NOT add assumptions or extend the summary; use ONLY what is written in the row.\n\n"

    "OUTPUT FORMAT REQUIREMENT (MANDATORY):\n"
    "Your response MUST contain exactly two lines of text:\n"
    "Mapped Sections: <comma-separated section names>\n"
    "Decision Logic: <short explanation referencing ONLY the given dataset and rule table>\n\n"

    "Example expected output:\n"
    "Mapped Sections: Section 15\n"
    "Decision Logic: Status indicates the signal is still under evaluation. "
    "According to ICH E2C(R2), ongoing signals are only listed in Section 15.\n\n"

    "JSON OUTPUT FORMAT (in addition to the two-line output):\n"
    "{\n"
    "  \"signal\": \"<signal term>\",\n"
    "  \"scenario\": \"<one of the seven scenarios>\",\n"
    "  \"mapped_sections\": [\"Section 15\", \"Section 16.2\", ...],\n"
    "  \"categorization_summary\": \"One-sentence classification summary. Must use ONLY dataset information.\",\n"
    "  \"decision_logic\": \"Full reasoning using rule-based logic; must NOT hallucinate or infer beyond input.\"\n"
    "}\n\n"

    "Now read the input row and generate BOTH the two-line output AND the JSON output."
)

In [ ]:
import time
# Loop through your signal terms
results = []
for _, row in df.iterrows():
    signal_info = (
        f"Signal: {row['Signal term']}\n"
        f"Date detected: {row['Date detected']}\n"
        f"Status: {row['Status']}\n"
        f"Date closed: {row['Date closed']}\n"
        f"Source of signal: {row['Source of signal']}\n"
        f"Summary: {row['Summary']}\n"
        f"Action: {row['Action']}"
    )
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": f"signal_info: {signal_info}"}
    ]
    start_time = time.time()
    response = llm.create_chat_completion(
        messages=messages,
        max_tokens=300,
        temperature=0.4
    )
    end_time = time.time()
    gen_time = round(end_time - start_time, 2)
    usage = response.get("usage", {})
    total_tokens = usage.get("total_tokens", None)
    tokens_per_sec = round(total_tokens / gen_time, 2)
    # Extract model reply
    reply = response["choices"][0]["message"]["content"].strip()
    row_result = row.to_dict()
    row_result["Mapping"] = reply
    row_result["Gen Time"] = gen_time
    row_result["Tokens Per Sec"] = tokens_per_sec
    results.append(row_result)

llama_perf_context_print:        load time =   52836.83 ms
llama_perf_context_print: prompt eval time =   52835.98 ms /   696 tokens (   75.91 ms per token,    13.17 tokens per second)
llama_perf_context_print:        eval time =   95457.85 ms /   145 runs   (  658.33 ms per token,     1.52 tokens per second)
llama_perf_context_print:       total time =  148692.46 ms /   841 tokens
llama_perf_context_print:    graphs reused =        139
Llama.generate: 656 prefix-match hit, remaining 46 prompt tokens to eval
llama_perf_context_print:        load time =   52836.83 ms
llama_perf_context_print: prompt eval time =    3524.64 ms /    46 tokens (   76.62 ms per token,    13.05 tokens per second)
llama_perf_context_print:        eval time =  155616.24 ms /   236 runs   (  659.39 ms per token,     1.52 tokens per second)
llama_perf_context_print:       total time =  159809.25 ms /   282 tokens
llama_perf_context_print:    graphs reused =        227
Llama.generate: 656 prefix-match hit, remaini

In [ ]:

from google.colab import files
# Save results
mapped_df = pd.DataFrame(results)
mapped_df.to_excel("mapped_signals_gemma.xlsx", index=False)
files.download("mapped_signals_gemma.xlsx")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>